# <center><b>Reinforcement Learning for Quadruped Locomotion</b></center>

<div align="center">

<h3>KAIST DRCD Lab</h3>


<b>Instructor</b><br>
<a href="https://dynamicrobot.kaist.ac.kr/people.html">Hae-Won Park</a>
(haewonpark [at] kaist.ac.kr)

<br>

<b>Teaching Assistants</b><br>
<a href="https://kdyun0118.github.io/">Dongyun Kang</a>
(kdong7309 [at] kaist.ac.kr)<br>
<a href="https://github.com/parkjahun42">Jaehyun Park</a>
(parkjahun42 [at] kaist.ac.kr)<br>
<a href="https://github.com/sowoolee">Sowoo Lee</a>
(dlthdn [at] kaist.ac.kr)


<video width="900" controls autoplay loop>
  <source src="../images/dummy_video.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

<br>

<img src="../images/dummy_image.png" width="900">




</div>

<br>
<br>

## Abstract

이 튜토리얼은 **MuJoCo 시뮬레이터 환경에서 사족보행 로봇(Go1)의 강화학습을 수행**하는 것을 목표로 합니다.
PPO(Proximal Policy Optimization)를 기반으로 한 보행 정책 학습 과정을 단계별로 설명하며,

- Go1 MuJoCo 환경 구성 및 관측/행동 설계
- PPO 학습 파이프라인과 주요 하이퍼파라미터
- 사전 학습된 정책(pretrained policy)을 활용한 파인튜닝
- Reward ablation을 통한 보행 성능 비교 분석

을 포함합니다.

본 튜토리얼은 **강화학습 기반 로봇 제어를 처음 접하는 학생부터, 실제 보행 정책을 분석하고자 하는 연구자**까지를 대상으로 설계되었습니다.

<br>

<center>

**세부 실험 구성 및 확장 가이드는 노트북 하단에 제공됩니다.**

</center>

<br>

---

## 0. Environment Setup

이 섹션에서는 튜토리얼 실행을 위한 환경을 설정합니다.

<div align="center">
  <img src="https://drive.google.com/uc?id=1tamjiJCNrQ5W-3KFr6JAoILyXDLMHoPQ"
       width="600">
</div>

본 튜토리얼은 **Google Colab** 환경을 기준으로 작성되었습니다.
아래 순서에 따라 실행 환경을 준비해 주세요.

1. **런타임 유형을 GPU(T4)로 변경**하고 런타임에 연결합니다.
2. 제공된 GitHub 레포지토리를 **clone**하여 코드 베이스를 준비합니다.
3. MuJoCo 및 강화학습 실험에 필요한 **의존성 패키지들을 설치**합니다.


In [ ]:
# Clone repository
import os, sys

import yaml

os.chdir("/content")
if not os.path.isdir("RL_DEMO"):
  !git clone https://github.com/DrcdKAIST/RL_DEMO.git
else:
  print("Cloned Directory already exists")

os.chdir("/content/RL_DEMO")
print("Current Directory: ", os.getcwd())

sys.path.insert(0, "/content/RL_DEMO")
os.environ["MUJOCO_GL"] = "egl"

In [ ]:
# Install dependencies
!pip install torch numpy tensorboard gymnasium==0.29.1 stable-baselines3==2.3.0 mujoco==3.1.5 imageio

---

<br>

## 1. Go1 MuJoCo Environment Description

본 튜토리얼에서는 **MuJoCo 기반 Go1 사족보행 로봇 환경**에서 강화학습을 수행합니다.

학습 환경은 `Go1MujocoEnv` 클래스로 구현되어 있으며, 로봇이 지정된 이동 command를 안정적으로 추종하는 과제를 **마르코프 결정 과정(Markov Decision Process, MDP)** 으로 구성합니다.

<br>

### Simulation Setup

- **물리 시뮬레이터**: MuJoCo
- **로봇 모델**: Unitree Go1
- **시뮬레이션 정의**:
  시뮬레이션 환경은 `unitree_go1/scene_position.xml` 파일에 정의되어 있으며,
  로봇 모델, 지면, 조명, 카메라 설정을 포함합니다.
- **로봇 설정**:
  관절 액추에이터, PD 제어 이득, 기구학 구조는 `go1_position.xml` 파일에 정의되어 있습니다.

<br>

### Environment Configuration

강화학습 환경의 로직은 다음 파일들에 구현되어 있습니다.

- `src/go1_mujoco_env.py` — 환경의 동역학 및 MDP 구성 로직
- `src/envs.yaml` — 환경 설정 파일로, 다음 항목들을 포함합니다.
  - 에피소드 길이
  - Reward 가중치
  - 목표 속도(command) 범위
  - Observation 스케일링
  - 조기 종료(early termination) 조건

<br>


In [ ]:
# Environment configuration
!sed -n '1,200p' src/envs.yaml


### Environment Step Loop

각 타임스텝마다 다음과 같은 절차가 수행됩니다.

1. 정책(policy)이 현재 상태를 기반으로 행동(action)을 출력합니다.
2. 시뮬레이터가 고정된 프레임 수(`frame_skip`)만큼 진행됩니다.
3. Observation, reward, 종료 조건이 계산됩니다.
4. 해당 전이(transition)가 학습을 위해 저장됩니다.

<br>

### Observation Space

Observation 벡터는 다음과 같은 정보를 포함합니다.

- 로봇 베이스의 선형 및 각속도
- 중력 방향이 투영된 벡터
- 목표 이동 속도(command)
- 관절 위치 및 속도
- 이전 타임스텝의 action (제어 입력의 연속성 확보 목적)

모든 observation은 학습 안정성을 위해 사전에 정의된 범위로 스케일링 및 클리핑됩니다.

<br>

### Reward and Termination

Reward 함수와 종료 조건은 다음 모듈에 분리되어 구현되어 있습니다.

- `src/mdp/reward.py`
- `src/mdp/termination.py`

이와 같은 모듈화된 구조를 통해 개별 reward 항을 손쉽게 수정하거나 제거할 수 있으며,
이는 튜토리얼 후반부에서 수행할 **reward ablation 실험**에서 활용됩니다.

In [ ]:
# Check observation / action space

import importlib
import numpy as np
import src.go1_mujoco_env as go1_env

importlib.reload(go1_env)

# Create environment (no rendering)
env = go1_env.Go1MujocoEnv(
    prj_path="/content/RL_DEMO",
    render_mode=None,
)

obs, info = env.reset()

print(f"Observation shape: {np.array(obs).shape}\n")
print(f"Action space: {env.action_space}\n")
print(f"Observation space: {env.observation_space}\n")

# Take one random step
action = env.action_space.sample()
obs, reward, terminated, truncated, info = env.step(action)

print("One-step reward:", reward)
print("Terminated:", terminated)

env.close()

In [ ]:
import numpy as np
import importlib

import os
import gc
import time
import imageio
import torch

from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback, CallbackList
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
from IPython.display import Video, display
from pathlib import Path

import src.go1_mujoco_env as go1_env

from src.utils.reward_logging_callback import RewardLoggingCallback

DEFAULT_CAMERA_CONFIG = {
    "azimuth": 90.0,
    "distance": 3.0,
    "elevation": -25.0,
    "lookat": np.array([0., 0., 0.]),
    "fixedcamid": 0,
    "trackbodyid": -1,
    "type": 2,
}

policy_cfg_path = Path("/content/RL_DEMO/src/params.yaml")
with policy_cfg_path.open("r", encoding="utf-8") as f:
    policy_cfg = yaml.safe_load(f)

---

<br>

## 2. PPO Code Review

### 2.1 Stable-Baselines3

본 튜토리얼에서는 **Stable-Baselines3(SB3)** 라이브러리에 구현된
**PPO(Proximal Policy Optimization)** 알고리즘을 사용합니다.

SB3는 PyTorch 기반의 강화학습 라이브러리로,
PPO, SAC, TD3 등 다양한 알고리즘을 안정적으로 제공합니다.

PPO는 actor-critic 구조를 기반으로 하며,
정책 업데이트 시 변화 폭을 제한(clipping)하여
학습 안정성을 확보하는 것이 특징입니다.

이 튜토리얼에서는 PPO의 수식적 유도보다는,
SB3에서 PPO가 어떻게 사용되는지에 초점을 맞춥니다.

<br>

### 2.2 Vectorized Environments

PPO는 on-policy 알고리즘이므로,
매 업데이트마다 많은 샘플을 필요로 합니다.

이를 위해 SB3는 여러 개의 환경을 동시에 실행하는
**vectorized environment**를 지원합니다.

본 튜토리얼에서는 `SubprocVecEnv`를 사용하여
여러 개의 Go1 환경을 병렬로 실행합니다.
이는 MuJoCo 기반 시뮬레이션에서 샘플 수집 속도를
크게 향상시켜 줍니다.

<br>

### 2.3 PPO 에이전트 생성

이제 병렬 환경을 기반으로 PPO 에이전트를 생성합니다.

본 튜토리얼에서는 두 가지 학습 방식을 지원합니다.

- 사전 학습된 policy를 불러와 추가 학습(finetuning)
- 네트워크를 새로 초기화하여 처음부터 학습

PPO의 정책 네트워크(actor)와 가치 함수(critic)는
SB3 내부에서 자동으로 생성됩니다.

<br>

### 2.4 PPO 학습 루프

PPO 에이전트가 생성되면,
`learn()` 함수를 통해 학습을 수행합니다.

`learn()` 내부에서는 다음 과정이 반복됩니다.

1. 현재 정책으로 병렬 환경과 상호작용하며 rollout 수집
2. GAE를 이용한 advantage 계산
3. 정책 및 가치 함수 업데이트
4. 학습 로그 기록 및 콜백 실행

In [ ]:
import stable_baselines3
import stable_baselines3.ppo.ppo as ppo_module
import inspect, os

print("stable-baselines3 version:", stable_baselines3.__version__)
print("PPO module path:", os.path.abspath(ppo_module.__file__))
print("PPO class:", ppo_module.PPO)

<div align="center">
  <img src="https://drive.google.com/uc?id=1jm4dvSTDlkRLh4HcoiYA-y8PsIiRuhui">
</div>

## 3. Training and Finetuning

Training quadruped locomotion policies from scratch can be time-consuming.
Therefore, this tutorial uses a **pretrained policy** as a starting point and
performs additional training (finetuning) in the target environment.

- If `USE_PRETRAINED = True`, a pretrained checkpoint is loaded and training continues.
- If `USE_PRETRAINED = False`, the policy is initialized from scratch.

This approach significantly reduces training time while still allowing
reward design and environment settings to influence the learned behavior.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/RL_DEMO/logs

In [ ]:
importlib.reload(go1_env)

USE_PRETRAINED = True
PRETRAINED_MODEL_PATH = "/content/RL_DEMO/models/pretrained/final_model.zip"

# Train
MODEL_DIR = "models"
LOG_DIR = "logs"

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

vec_env = make_vec_env(
    go1_env.Go1MujocoEnv,
    env_kwargs={"prj_path": "/content/RL_DEMO"},
    n_envs=policy_cfg["n_envs"],
    seed=policy_cfg["seed"],
    vec_env_cls=SubprocVecEnv,
)

train_time = time.strftime("%Y-%m-%d_%H-%M-%S")
run_name = f"{train_time}"

model_path = f"{MODEL_DIR}/{run_name}"
print(
    f"Training on {policy_cfg['n_envs']} parallel training environments and saving models to '{model_path}'"
)

checkpoint_callback = CheckpointCallback(
    save_freq=policy_cfg["policy"]["n_steps"] * policy_cfg["log"]["interval"],  # e.g. 100_000
    save_path=model_path,  # directory
    name_prefix="model",  # checkpoint_model_100000_steps.zip
    save_replay_buffer=False,
    save_vecnormalize=False,
)

eval_callback = EvalCallback(
    vec_env,
    best_model_save_path=model_path,
    log_path=LOG_DIR,
    eval_freq=policy_cfg["eval_freq"],
    n_eval_episodes=5,
    deterministic=True,
    render=False,
)

reward_logging_callback = RewardLoggingCallback()

callbacks = CallbackList([
    eval_callback,
    checkpoint_callback,
    reward_logging_callback,
])

if USE_PRETRAINED:
    print(f"Using Pretrained model from {PRETRAINED_MODEL_PATH}")
    model = PPO.load(path=PRETRAINED_MODEL_PATH, env=vec_env)
    model.tensorboard_log = LOG_DIR
else:
    print("Training from Network model from scratch")
    model = PPO("MlpPolicy", vec_env, verbose=1, tensorboard_log=LOG_DIR)

model.learn(
    total_timesteps=policy_cfg["total_timestep"],
    reset_num_timesteps=True,
    progress_bar=True,
    tb_log_name=run_name,
    callback=callbacks,
)
# Save final model
model.save(f"{model_path}/final_model")

vec_env.close()

del model
del eval_callback
del vec_env

gc.collect()

## 4. Policy Evaluation

After training, the learned policy is evaluated in a single environment.
The robot’s behavior is rendered and saved as a video for qualitative analysis.

- Control frequency: 50 Hz
- Video frame rate: 25 FPS
- Frames are sampled periodically and saved as an MP4 file

The following cell generates and displays a rollout video.


In [1]:
# Test
importlib.reload(go1_env)
model_path = "/content/RL_DEMO/models/pretrained/final_model"

WIDTH, HEIGHT = 544, 368

env = go1_env.Go1MujocoEnv(
    prj_path="/content/RL_DEMO",
    render_mode="rgb_array",
    camera_name="tracking",
    width=WIDTH,
    height=HEIGHT,
)
inter_frame_sleep = 0.0

model = PPO.load(path=model_path, env=env, verbose=1)

video_path = "/content/rollout.mp4"

obs, _ = env.reset()
max_time_step_s = policy_cfg["test"]["max_time_step_s"]
ep_len = 0
video_fps = 25

# Ctrl Hz: 50
render_interval = 50 // video_fps
max_steps = int(max_time_step_s * 50)

writer = imageio.get_writer(
    video_path,
    fps=video_fps,
    codec="libx264",
    quality=8,
    pixelformat="yuv420p",
)

frames = []

while ep_len < max_steps:
  with torch.no_grad():
    action, _ = model.predict(obs, deterministic=True)
  obs, reward, terminated, truncated, info = env.step(action)

  if ep_len % render_interval == 0:
    frame = env.render()
    frames.append(frame)

  ep_len += 1

imageio.mimwrite(
    video_path,
    frames,
    fps=video_fps,
    codec="libx264",
    quality=8,
    pixelformat="yuv420p",
)

env.close()

print("Saved video to:", video_path)

display(
    Video(
        video_path,
        embed=True,
        html_attributes="controls autoplay loop"
    )
)

NameError: name 'importlib' is not defined

## 5. Reward Ablation Study

Finally, we examine how reward design affects locomotion behavior.

By loading checkpoints trained with different reward configurations
(e.g., removing smoothness or energy penalties), we can directly observe
how each reward term influences stability, motion smoothness, and failure modes.

In the following cells, we compare rollout videos from different checkpoints
under identical evaluation conditions.
